## Train baseline

In [51]:
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from product_classification.data_processing.text_processing import EStemTag, Lemmatizer
import pickle

In [52]:
path = "../data"
with open(f"{path}/datasets.pkl", "rb") as handle:
    simple_datasets = pickle.load(handle)

### Categorical features transformation

In [9]:
categorical_features = ["merchant_name", "brand_name"]
categorical_transformer = OneHotEncoder(drop="first" ,handle_unknown="ignore", dtype=int)

### Numerical features transformation

In [10]:
numeric_features = ["price"]
numeric_transformer = StandardScaler()

### Text transformation

In [11]:
text_features = "product_name"

stem_tag = EStemTag.STEMMER
lemma = Lemmatizer(stem=stem_tag)

text_transformer = Pipeline(
    [(stem_tag.value, lemma),
    ("tfidf" ,TfidfVectorizer())]
)

### Processor

In [12]:
preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
            ("text", text_transformer, text_features)
                     ]
       )

### Classifier

In [42]:
classifier = LGBMClassifier(objective="binary", random_state=42, silent=False, metric="binary_logloss")

In [43]:
multilabel_classifier = MultiOutputClassifier(classifier, n_jobs=-1)

### Pipeline

In [44]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", multilabel_classifier)]
)

### train

In [45]:
X_train, y_train = simple_datasets.training.iloc[:,1:5], simple_datasets.training.iloc[:,5:]

In [46]:
clf.fit(X_train, y_train, classifier__verbose=True)

/home/okteto/product_classification/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/home/okteto/product_classification/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/home/okteto/product_classification/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped t

/home/okteto/product_classification/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/home/okteto/product_classification/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/home/okteto/product_classification/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['price']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                dtype=<class 'int'>,
                                                                handle_unknown='ignore'),
                                                  ['merchant_name',
                                                   'brand_name']),
                                                 ('text',
                                                  Pipeline(steps=[('stemmer',
                                                                   Lemmatizer(stem=<EStemTag.STEMMER: 'stemmer'>)),
                                                                  ('tfidf',
                                                                   TfidfVectorizer())]),
                                                  'product_name')])),
                ('classifier',
                 MultiOutputClassifier(estimator=LGBMClassifier(metric='binary_logloss',
                                                                objective='binary',
                                                                random_state=42,
                                                                silent=False),
                                       n_jobs=-1))])

### Eval

In [47]:
X_test, y_test = simple_datasets.test.iloc[:,1:5], simple_datasets.test.iloc[:,5:]

In [48]:
y_pred = clf.predict(X_test)

/home/okteto/product_classification/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [49]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import classification_report

In [50]:
print(classification_report(y_test, y_pred, target_names=list(y_test.columns)))

                                    precision    recall  f1-score   support

                        animalerie       0.95      0.84      0.89       691
                      auto et moto       0.84      0.59      0.70       350
                   bagages et sacs       0.86      0.70      0.77       290
                  beaute et parfum       0.94      0.90      0.92       757
              bebe et puericulture       0.92      0.64      0.75       399
                            bijoux       0.96      0.88      0.92       361
                         bricolage       0.85      0.61      0.71       753
                     cd et vinyles       0.58      0.37      0.45        30
         chaussures et accessoires       0.94      0.87      0.90       703
    commerce, industrie et science       0.28      0.24      0.26        29
                 cuisine et maison       0.87      0.57      0.69       758
                    dvd et blu-ray       0.98      0.93      0.96       103
           

/home/okteto/product_classification/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped t